<a/ id='top'></a>

# CSCI4022 Homework 5; A-Priori

## Due Monday, March 13 at 11:59 pm to Canvas and Gradescope

#### Submit this file as a .ipynb with *all cells compiled and run* to the associated dropbox.

***

Your solutions to computational questions should include any specified Python code and results as well as written commentary on your conclusions.  Remember that you are encouraged to discuss the problems with your classmates, but **you must write all code and solutions on your own**.

**NOTES**: 

- Any relevant data sets should be available on Canvas. To make life easier on the graders if they need to run your code, do not change the relative path names here. Instead, move the files around on your computer.
- If you're not familiar with typesetting math directly into Markdown then by all means, do your work on paper first and then typeset it later.  Here is a [reference guide](https://math.meta.stackexchange.com/questions/5020/mathjax-basic-tutorial-and-quick-reference) linked on Canvas on writing math in Markdown. **All** of your written commentary, justifications and mathematical work should be in Markdown.  I also recommend the [wikibook](https://en.wikibooks.org/wiki/LaTeX) for LaTex.
- Because you can technically evaluate notebook cells is a non-linear order, it's a good idea to do **Kernel $\rightarrow$ Restart & Run All** as a check before submitting your solutions.  That way if we need to run your code you will know that it will work as expected. 
- It is **bad form** to make your reader interpret numerical output from your code.  If a question asks you to compute some value from the data you should show your code output **AND** write a summary of the results in Markdown directly below your code. 
- 45 points of this assignment are in problems.  The remaining 5 are for neatness, style, and overall exposition of both code and text.
- This probably goes without saying, but... For any question that asks you to calculate something, you **must show all work and justify your answers to receive credit**. Sparse or nonexistent work will receive sparse or nonexistent credit. 
- There is *not a prescribed API* for these problems.  You may answer coding questions with whatever syntax or object typing you deem fit.  Your evaluation will primarily live in the clarity of how well you present your final results, so don't skip over any interpretations!  Your code should still be commented and readable to ensure you followed the given course algorithm.
- There are two ways to quickly make a .pdf out of this notebook for Gradescope submission.  Either:
 - Use File -> Download as PDF via LaTeX.  This will require your system path find a working install of a TeX compiler
 - Easier: Use File ->  Print Preview, and then Right-Click -> Print using your default browser and "Print to PDF"



---
**Shortcuts:**  [Problem 1](#p1) | [Problem 2](#p2) | [Extra Credit](#p3) |
---


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
import itertools #may use for .combinations/similar, if desired.

***
<a/ id='p1'></a>
[Back to top](#top)
# Problem 1 (Practice: Candidate Items; 20 pts)

In the A-Priori algorithm, there is a step in which we create a candidate list of frequent itemsets of size $k+1$ as we prune the frequent itemsets of size $k$.  This this problem we will create two functions to do that formally.

#### Part A:

There are two types of data objects in which we might be holding the frequency counts of itemsets.  If $k=2$, they may be stored in a triangular array.  Create a function `Cand_Trips` that takes a triangular array and returns all valid candidate triples as a list.  Recall that the itemset $\{i,j,k\}$ is only a candidate if all 3 of the itemsets in $\{\{i,j\}, \{i,k\}, \{k,j\}\}$ are frequent.

Some usage notes:

- The first input argument is `triang_counts`,  a zero-indexed triangular (numeric) array, by same convention as introduced in class.
- The second input argument is the positive integer support threshold `s`.
- The underlying itemset is 0-indexed, so e.g. `[0,1,3]` is a valid triple.
- You should not convert the input list `triang_counts` into a list of triples as part of your function.
- The return array `candidates` should be a list of 3-index lists of the item numbers of the triples.  So a final answer for some input might be:

`cand_trips` =
    `[[0,3,4], [1,2,7]]`

- An implementation note: there are two fundamentally different ways to think about implementing this function.  Option 1 involves thinking about the elements of `triang_counts` in terms of their locations on the corresponding *triangular matrix*: scan row $i$ for a pair of frequent pairs $\{\{i,j\}, \{i,k\}\}$ and then check if $\{j,k\}$ is in fact frequent.  Option 2 scans all of `tri_Counts` for frequent item pairs (the "pruning" step) and saves those in some object with their indices, then scans *that* object for candidates.  Both are valid for this problem, but option 2 may generalize to higher $k$ better...

In [2]:
def tai(i,j,n):
    k = i*(n - (i+1)/2) + j - i - 1
    return int(k)

def isInList(myarr, list_arrays):
#     print(list_arrays)
#     print(myarr[0], myarr[1])
    for i in range(len(list_arrays)):
        if(list_arrays[i][0] == myarr[0] and list_arrays[i][1] ==myarr[1]):
            return True
    return False
def cand_trips(triang_counts, s):
    candidates = []
    
    #First step: find frequent pairs
    #Store as an array of arrays, indexed by the row of the triangular matrix
    freq = np.array([np.array([(i,j) for j in range(5) if i<j and triang_counts[tai(i,j,5)]>=s]) for i in range(5)],dtype=object)
    
    #Lets print this out so we can see what we're working with
#     print("\nFreq array:\n{}".format(freq))
    """Your Code Here"""
    #Loop over the rows of the freq array
    #Take a pair of pairs that could form a triple
    #For example, in case of s>=3 above, we could try to make a triple from the first 2 pairs in row 1: [0,1] & [0,2]
    #Before we can add a candidate triple ([0,1,2]), we need to check if the required third pair ([1,2]) meets the support threshold
    #For example, we already know that [0,1] & [0,2] are frequent pairs, but what about the third pair [1,2]?
    #So, you need to check that the triang_counts value at [1,2], and if it meets support threshold, you can add the trip [0,1,2] to your candidates 
    #Then you'll repeat, next time considering [0,1] & [0,3], then [0,2] & [0,3]
    #Move through all the rows of freq doing this (ie, you'll next move to consider the 2nd row of freq, the pairs [1,2] & [1,3] etc.)
    freq_flat=[]
#     print(freq)
    for i in range(len(freq)):
        for j in range(len(freq[i])):
            freq_flat.append(freq[i][j])
#     print("FREQ_FLAT",freq_flat)
#     print(isInList([1,2],freq_flat))
    for i in range(len(freq_flat)-1):
        for j in range(i, len(freq_flat)):
            if(freq_flat[i][0] == freq_flat[j][0]):
                v=[freq_flat[i][1], freq_flat[j][1]]
    #             print(v)
                if(isInList(v,freq_flat)):
                    candidates.append([freq_flat[i][0], freq_flat[i][1], freq_flat[j][1]])
#     print(freq)
#     print("Freq[0]:", freq[0])
#     print("Flatten:",freq.flatten().flatten().flatten())
    return candidates

#### Part B:

A quick test case.  Below is  a matrix $M$ and code including its corresponding the triangular array.  

$C=\begin{bmatrix}
\cdot &10&7&3&2\\
\cdot &\cdot&6&4&3\\
\cdot &\cdot&\cdot&3&6\\
\cdot &\cdot&\cdot&\cdot&0\\
\cdot &\cdot&\cdot&\cdot&\cdot\\
\end{bmatrix}$
 
Input the given list into your function to verify that it returns the correct valid triples at $s=1$ and $s=6$.

In [3]:
triang_counts=[10,7,3,2,6,4,3,3,6 ,0]
print('For s>=6, candidate:', cand_trips(triang_counts, 6))
print('For s>=1, candidates:', cand_trips(triang_counts, 1))

#Check that...
#cand_trips(triang_counts, 1) returns all the possible triples except those that contain BOTH items 3 and 4.
#cand_trips(triang_counts, 6) returns only the triple [[0,1,2]]

For s>=6, candidate: [[0, 1, 2]]
For s>=1, candidates: [[0, 1, 2], [0, 1, 3], [0, 1, 4], [0, 2, 3], [0, 2, 4], [1, 2, 3], [1, 2, 4]]


#### Part C:

Suppose instead that our $k=2$ item counts were stored in a list of the form e.g.
`pairs_counts` =
    `[[0,1,12], [0,2,0], [0,3,11], ..., [7,8,103]]`
    
Where each element is a triple storing the two item indices and their count, $[i,j,c_{ij}]$. 

Create a function `cand_trips_list` that takes in a list of pairs counts and returns all valid candidate triples as a list.  

Some usage notes:

- The first input argument is `pairs_counts`,  a zero-indexed list of triples.
- The second input argument is the positive integer support threshold `s`.
- The underlying itemset is 0-indexed, so e.g. `[0,1,3]` is a valid triple.
- The return array `candidates` should be a list of 3-element lists, as above.

You should **not** convert the input list `pairs_counts` into a triangular array as part of your function.  After all, sometimes we use the list format for pairs because it saves memory compared to the triangular array format!  You may be able to borrow heavily from the logic of your first function, though!

In [4]:
def cand_trips_list(pairs_counts, s):
    pairs_above_support = []
    candidates=[]
    for i in pairs_counts:
        if(i[2]>=s):
            pairs_above_support.append(np.array([i[0],i[1]]))
    for j in range(len(pairs_above_support)):
        for l in range(j+1, len(pairs_above_support)):
            if(pairs_above_support[j][0] == pairs_above_support[l][0]):
#                 print(pairs_above_support[j])
                if(isInList(np.array([pairs_above_support[j][1], pairs_above_support[l][1]]), pairs_above_support)):
                    candidates.append(np.array([pairs_above_support[j][0], pairs_above_support[j][1], pairs_above_support[l][1]]))
    return candidates


#### Part D:

Do the test case again.  Below is the list reprentation of the same matrix $M$ from part B.  
 
Input the given list into your function to verify that it returns the correct valid triples at $s=1$ and $s=6$.

In [5]:
pairs_counts=[[0,1,10], [0,2,7], [0,3,3], [0,4,2],\
             [1,2,6],[1,3,4], [1,4,3],\
             [2,3,3],[2,4,6],\
             [3,4,0]]
# print(pairs_counts)
print(cand_trips_list(pairs_counts, 6))
print(cand_trips_list(pairs_counts, 1))
#Check that...
#cand_trips(pairs_counts, 1) returns all the possible triples except those that contain BOTH items 3 and 4.
#cand_trips(pairs_counts, 6) returns only the triple [[0,1,2]]

[array([0, 1, 2])]
[array([0, 1, 2]), array([0, 1, 3]), array([0, 1, 4]), array([0, 2, 3]), array([0, 2, 4]), array([1, 2, 3]), array([1, 2, 4])]


#### Part E

Describe *in words* how you would generalize your code in part D to work for generating candidate quadruples $[i_1, i_2, i_3, i_4]$ from an input list of triples counts (each element of the form $[i, j, k, c_{ijk}]$).

To find a candidate quadruple from a list of triples the code would start the same. I would find all triples that have a count higher than the support threshold and add them to a list. Then I would iterate through the list. On each member of the list I would check if the first two elements match another member then check if there exists a member of the list where the first two elements are the last two elements of the first member and the last element is the last element of the second member.

[0,1,2], [0,1,3] search for [1,2,3] and if all exist then[0,1,2,3] is a valid quadruple.


***
<a/ id='p2'></a>
[Back to top](#top)
# Problem 2 (Practice: A-Priori; 25 pts) 

Consider the recipe data set provided in `recipes.npy` (use `np.load`).  This includes 100,000 recipes from a variety of sources.

We want to use the baskets and the ingredients therein (see `ingredients.npy`) to perform an item basket analysis.

This data set is small enough to run directly from main memory, so you may do that if you wish.

Loading and accessing the data set is shown below:

In [6]:
# '../recipes.npy'  '../ingredients.npy'

recipes=np.load('../recipes.npy', allow_pickle=True)
ingredients=np.load('../ingredients.npy', allow_pickle=True)

In [7]:
print(recipes[:2]) #list of lists
print(ingredients[:2]) #inventory list
print(ingredients[recipes[0]]) #to access a recipe by string

# print(ingredients[:])

[array([ 233, 2754,   42,  120,  560,  345,  150, 2081,   12,   21],
       dtype=int64)
 array([ 198,  249,    2,  194, 1884,  791,  965,  423,   53,   48,  798,
          31,  362, 1031,   94,   26,    8], dtype=int64)                ]
['salt' 'pepper']
['basil leaves' 'focaccia' 'leaves' 'mozzarella' 'pesto' 'plum tomatoes'
 'rosemary' 'sandwiches' 'sliced' 'tomatoes']




#### a) Since the ingredients file alrady provides integer codes for each of our items, we can move directly into countin via the A-Priori algorithm.  Using the two given files, create a table of frequent single items at 1% support threshold. You may use Python's native classes to set up your lookup functions/tables.

Was 1% an appropriate support threshold?  Describe why or why not.  Keep in mind, the goal here is two fold: you want "actionable" conclusions, and output that's small enough that you or your grader can make sure that you have the right set!


In [8]:
counts=[0] * len(ingredients)
support = 0.20 * len(ingredients)
L1 = []
for r in recipes: #counting step
    for i in r:
        counts[i] +=1
new_lookup={}
freq_lookup={}
ident = 0
for q in range(len(counts)): #pruning step
    if(counts[q]<support):
        counts[q] = -1
    else:
        new_lookup[ingredients[q]] = ident
#         freq_lookup[ingredients[q]] = counts[q]
        L1.append(ingredients[q])
        ident+=1
# print(new_lookup)

The 1% support threshold of 35 was too much for my computer so I raised it to 5% which was still too much. I would run the code, wait 5 minutes then stop it and increase the support percentage threshold. I ended on 20% because my computer could run the script in less than 10 minutes. My code was probably not optimized in such a way that would allow my computer to find the number of pairs for bigger data sets. Additionally my computer is not very strong.


#### b) Use A-priori to find all frequent  pairs of items from your set of frequent items in a).  Use whatever support threshold you feel is most appropriate, but make sure your result is readable: you should list the top handful of most frequent pairs, sorted by their prevalence.

Report the confidences of the two association rules corresponding to the most frequent item pair.


In [9]:
trips=[] #initiate trips array
for idx1 in range(len(L1)):
    for idx2 in range(idx1+1, len(L1)):
#         print(new_lookup[L1[idx1]])
        trips.append((new_lookup[L1[idx1]], new_lookup[L1[idx2]], 0))

In [10]:
# used nb09
for recipe in recipes:
    frequent_items = list(set(ingredients[recipe])&set(L1)) #creates set of strings of recipes
    for idx1 in range(len(frequent_items)):
        for idx2 in range(idx1+1, len(frequent_items)):
            new_idx1 = new_lookup[frequent_items[idx1]] #
            new_idx2 = new_lookup[frequent_items[idx2]]
            if new_idx1 > new_idx2: # makes sure the order is correct
                new_idx1, new_idx2 = new_idx2, new_idx1
            k = int(new_idx1*(len(L1) - (new_idx1+1)/2) + new_idx2 - new_idx1 - 1) #same as tai but 
            trips[k] = (new_idx1, new_idx2, trips[k][2]+1)

In [11]:
pairs_above_support = [] #used code from 1-C
candidates=[]
for i in trips:
    if(i[2]>=support): #checks if frequencies are above support 
        pairs_above_support.append(np.array([i[0],i[1]]))
for j in range(len(pairs_above_support)):
    for l in range(j+1, len(pairs_above_support)):
        if(pairs_above_support[j][0] == pairs_above_support[l][0]):
#                 print(pairs_above_support[j])
            if(isInList(np.array([pairs_above_support[j][1], pairs_above_support[l][1]]), pairs_above_support)):
                candidates.append(np.array([pairs_above_support[j][0], pairs_above_support[j][1], pairs_above_support[l][1]]))

In [12]:
print(candidates)

[array([0, 1, 2]), array([0, 1, 3]), array([0, 1, 4]), array([0, 1, 5]), array([0, 1, 6]), array([0, 1, 7]), array([0, 1, 8]), array([0, 1, 9]), array([ 0,  1, 10]), array([ 0,  1, 11]), array([ 0,  1, 12]), array([ 0,  1, 13]), array([ 0,  1, 14]), array([ 0,  1, 15]), array([ 0,  1, 16]), array([ 0,  1, 17]), array([ 0,  1, 18]), array([ 0,  1, 19]), array([ 0,  1, 20]), array([ 0,  1, 21]), array([ 0,  1, 23]), array([ 0,  1, 24]), array([ 0,  1, 25]), array([ 0,  1, 26]), array([ 0,  1, 27]), array([ 0,  1, 28]), array([ 0,  1, 29]), array([ 0,  1, 30]), array([ 0,  1, 31]), array([ 0,  1, 32]), array([ 0,  1, 33]), array([ 0,  1, 35]), array([ 0,  1, 39]), array([ 0,  1, 40]), array([ 0,  1, 42]), array([ 0,  1, 44]), array([ 0,  1, 45]), array([ 0,  1, 46]), array([ 0,  1, 47]), array([ 0,  1, 49]), array([ 0,  1, 50]), array([ 0,  1, 51]), array([ 0,  1, 54]), array([ 0,  1, 55]), array([ 0,  1, 56]), array([ 0,  1, 57]), array([ 0,  1, 58]), array([ 0,  1, 59]), array([ 0,  1, 

**c)**

Zach has to go to the store and stock his pantry.  He knows that his girlfriend has a (borderline unhealthy to those around her) love of garlic.  What should he purchase to make sure he has in stock?  What are two most frequent $\{garlic, x\}$ item pairs, and what are the two most **interesting** $garlic \to X$ associations?

In [28]:
# finding garlic
# for i in range(len(ingredients)):
#     if(ingredients[i]=='garlic'):
#         print(i)
# i = 3
# print(new_lookup.values())
gar_combos = {}
gar_cands=[]
for c in candidates:
    if(c[0]==3 or c[1]==3 or c[2]==3):
        gar_cands.append(c)
for a in new_lookup.values():
    gar_combos[a] = 0
for c in gar_cands:
    if(c[0] != 3):
        gar_combos[c[0]] = gar_combos[c[0]] + 1
    if(c[1] != 3):
        gar_combos[c[1]] = gar_combos[c[1]] + 1
    if(c[2] != 3):
        gar_combos[c[2]] = gar_combos[c[2]] + 1
maxx=0
maxx2=0
maxx_idx=-1
maxx2_idx=-1
for i in new_lookup.values():
    if(gar_combos[i]>maxx):
        maxx=gar_combos[i]
        maxx_idx=i
    elif(gar_combos[i]>maxx2):
        maxx2=gar_combos[i]
        maxx2_idx=i
print(maxx, maxx_idx)
print(maxx2, maxx2_idx)

print(ingredients[maxx_idx])
print(ingredients[maxx2_idx])

171 1
120 6
pepper
onion


The two most frequent garlic pairs are pepper and onion. He should purchase those!

***
<a/ id='p3'></a>
[Back to top](#top)
# Problem 3 (Extra-Credit: A-Priori with hashing and more baskets; 10 pts each part) 

The data set in 2 had two very appealing propeties that we typically do **not** assume to be the case:
- It came with an ingredient list provided
- It was small enough to fit into main memory.

To fully implement the model, you can get some extra credit by attempting variants of the data that do not have those properties.  We will tackle each problem individually.  You should answer each problem *in its own, separate notebooks* to ensure you're not using any variables from your solution to problem 2 above.

## EC1: A-P with hashing

#### EC1a) The file `recipesbying` contains the same data set as in problem 2, but the strings themselves live in each recipe.

Create a hash table as in nb08 that hashes each ingredient observed based on its string. In other words, create your own version of what **was** in `ingredients.npy` by creating your own hash and/or lookup functions.
Include a check to minimize and fix any collisions, as in nb08.



**EC1b)** Use A-priori to find all frequent items and all frequent pairs of items from your hashed data set in part EC1).  Ensure that the results match those of problem 2.



## EC2: A-P with massive data

The `.npz` file `simplified-recipes-1M.npz` contains over 1 million recipes, and is the original source of the 100,000 recipes used in problem 2.  Using this file (and `ingredients.npy`, if desired), use A-priori to find all frequent items and all frequent pairs of items.  However, you should **not** load all of the file into main memory.  Instead, use `np.memmap` or other options to ensure that you never load into main memory more than 100,000 recipes at a time.  Include any processing in your submission, and use the same proportionate support threshold as you did in problem 2.  Do the most common items differ?

A few notes: 
- If you process the data to make it readable in other forms `.npy`, `.csv`, etc., that's fine, but show all processing code in your submission.
- For example, if you find `.memmap` hard to get working, you may convert to `.csv` and use `pd.read_csv` with arguments `chunksize` or `skiprows`, `nrows`
- You may be able to do the problem with very little additional work if you are clever about how you open the file and read over it.  In this case, set up your "loop" over baskets to only go over 100,000 rows of the file at a time, though, and be very explicit as to how you're avoiding the larger objects ever entering main memory.